In [1]:
import pandas as pd
from pathlib import Path


# helpers 

In [35]:
src_path = Path('/mnt/c/Users/umarh/Dropbox/Work/Employment/Shopify Transformation - Nabil')

In [36]:
raw = src_path.joinpath('raw_data')
dim = src_path.joinpath('dimension_lookups')
output = src_path.joinpath('output')

In [50]:
pd.to_datetime(list(src_path.glob('*.csv'))[0].stat().st_mtime,unit='s')

Timestamp('2021-09-11 17:09:49.447462656')

In [54]:
def get_newest_file(path : Path, ext : str) -> Path:
    
    file_d = {
                  f : 
                  pd.to_datetime(f.stat().st_mtime, unit='s') 
              for f 
              in path.glob(f'*.{ext}')
             }
    return max(file_d, key =file_d.get) 

# Sample Output



In [38]:
samp = pd.read_csv(src_path.joinpath('Result.csv'))

In [56]:
get_newest_file(raw, 'xls')

PosixPath('/mnt/c/Users/umarh/Dropbox/Work/Employment/Shopify Transformation - Nabil/raw_data/8-21 stock list.xls')

# Raw Data

In [58]:
raw_df = pd.read_excel(get_newest_file(raw, 'xls'))

In [74]:
# remove total column, has chinese character in so will use interger slicing. 
raw_df = raw_df.iloc[:,:-1]

In [79]:
# chinese char again so rename using list slicing.
raw_df = raw_df.rename(columns={raw_df.columns[0] : 'SKU'})

In [102]:
raw_transform = pd.melt(raw_df,
        id_vars='SKU',
        var_name='Option1 Value', 
        value_name='70 rue de la prulay').dropna(subset=['70 rue de la prulay']
        )

# Lookups 

In [133]:
dim_df = pd.read_excel(get_newest_file(dim,'xlsx'),engine='openpyxl')

In [134]:
dim_df['SKU_Cleaned'] = dim_df['SKU'].str.split('-',expand=True)[0]

In [135]:
dim_df_sku_handl = dim_df.drop_duplicates(subset=['SKU_Cleaned', 'Handle'])[['Handle','SKU_Cleaned']]

In [136]:
# dim_df_sku_handl = dim_df_sku_handl.rename(columns={'SKU_Cleaned': 'SKU'})

In [165]:
result = pd.merge(raw_transform.assign(SKU_Cleaned=
                              raw_transform['SKU']\
                              .str.replace('\(.*\)','',regex=True)
                             ), 
         dim_df_sku_handl,
         on=['SKU_Cleaned']
         ,how='left')

In [166]:
result['Option1 Name'] = 'Size'

In [167]:
#assign missing columns
result = result.assign(**{s : pd.NA for s in samp.columns if not s in result.columns})

In [168]:
raw_transform.shape

(3619, 3)

In [169]:
result_final = result[samp.columns]

In [170]:
pd.Timestamp('today').strftime('%Y%m%d_result.csv')

'20210911_result.csv'

In [176]:
result_final = result_final.sort_values(['SKU'])

In [177]:
result_final.to_csv(
    output.joinpath(pd.Timestamp('today').strftime('%Y%m%d_%H%M%S_result.csv')
                   ), index=False)